In [4]:
import time
import re
import sys
import requests
import numpy as np
import pandas as pd
from pathlib import Path

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# Uber Eats Scraping

In [5]:
url_uber = 'https://www.ubereats.com/mx/feed?ps=1&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMkNhdGVkcmFsJTIwTWV0cm9wb2xpdGFuYSUyMGRlJTIwbGElMjBDaXVkYWQlMjBkZSUyME0lQzMlQTl4aWNvJTIyJTJDJTIycmVmZXJlbmNlJTIyJTNBJTIyMzNjOWJiZGItNWY3OC01MGY2LTdmNmMtNDVmNzY4MzEzOTc5JTIyJTJDJTIycmVmZXJlbmNlVHlwZSUyMiUzQSUyMnViZXJfcGxhY2VzJTIyJTJDJTIybGF0aXR1ZGUlMjIlM0ExOS40MzAxNDQ2JTJDJTIybG9uZ2l0dWRlJTIyJTNBLTk5LjEzMzkxODYlN0Q%3D'

driver = webdriver.Chrome()

driver.get(url_uber)

In [6]:
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)
button = driver.find_element_by_xpath('//button[normalize-space()="Mostrar más"]')
ActionChains(driver).click(button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)
button = driver.find_element_by_xpath('//button[normalize-space()="Mostrar más"]')
ActionChains(driver).click(button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()

page_source = driver.page_source

In [7]:
driver.quit()

Uber = BeautifulSoup(page_source, 'lxml')

In [73]:
#print(Uber.prettify())

In [74]:
#get store names

store_names = Uber.select('h3')

store_names = [name.text for name in store_names]

In [79]:
# get ratings

ratings = Uber.find_all('div', class_='gg cc cd ec eb j7 cf g8 ag c0 g4')

ratings = [rating.text for rating in ratings]

In [76]:
# get mashup list of delivery fee & delivery time

mash = Uber.find_all('div', class_='cb f4 cr j5')

mash = [element.text for element in mash]


# filter delivery fee
delivery_fee = []

i= 1

for x in range(int(len(mash)/5)):
    delivery_fee.append(mash[i])
    i+=3
    

# filter delivery times
delivery_time = []

i = 2
for x in range(int(len(mash)/5)):
    delivery_time.append(mash[i])
    i+=3


In [89]:
uber_data = pd.DataFrame(list(zip(store_names,ratings,delivery_fee,delivery_time)),columns = ['Store','Rating','Delivery Fee','Delivery Time'])

uber_data

,Store,Rating,Delivery Fee,Delivery Time
0,Di Paulos (Centro),4.4,Costo de envío: MXN15,25–35 min
1,Restaurante Zesta Punta,4.5,Costo de envío: MXN21,30–40 min
2,Dynasty (PORTAL CENTRO),4.2,Costo de envío: MXN17,30–40 min
3,alas k,4.6,Costo de envío: MXN21,30–40 min
4,Taqueria el hermanito,4.2,Costo de envío: MXN19,40–50 min
...,...,...,...,...
67,Súper Tacos CDMX,4.7,Costo de envío: MXN27,30–40 min
68,Caribe Venezuelan Kitchen,4.2,Costo de envío: MXN23,30–40 min
69,ZEMPO SUSHI,4.6,Costo de envío: MXN21,20–30 min
70,El Antojito,4.6,Costo de envío: MXN17,35–45 min


In [107]:
uber_data.to_csv("/Users/miguel.d.ferrusca/Google Drive/Ironhack/Projects/web_scrapping/uber_data.csv")

# Rappi

In [12]:
url_rappi = 'https://www.rappi.com.mx/restaurantes'

driver = webdriver.Chrome()

driver.get(url_rappi)

In [13]:
popup_button = driver.find_element_by_xpath('//*[@id="address-modal"]/div/div/div[1]/div[2]/div[2]/div/div/div[1]/div[1]/i')
ActionChains(driver).click(popup_button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)
more_button = driver.find_element_by_xpath('//*[@id="app-container"]/div/span/app-by-stores-home/div[1]/div[2]/div[2]/button')
ActionChains(driver).click(more_button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()

page_source = driver.page_source

In [14]:
driver.quit()

In [15]:
rappi_soup = BeautifulSoup(page_source,'lxml')

In [17]:
rappi_stores = rappi_soup.select('h2', class_='store-name')
rappi_stores_names = [store.text for store in rappi_stores]

In [99]:
rappi_cuisines = rappi_soup.select('h3',class_='store-tags')
rappi_cuisines_names = [cuisine.text for cuisine in rappi_cuisines]

rappi_cuisines_names = rappi_cuisines_names[:80]

In [100]:
rappi_rating = rappi_soup.find_all('span', class_='score ng-star-inserted')

rappi_ratings = [star.text for star in rappi_rating]

In [101]:
rappi_eta = rappi_soup.find_all('span', class_='store-eta ng-star-inserted')

rappi_eta = [eta.text for eta in rappi_eta]
len(rappi_eta)

80

In [103]:
rappi_data = pd.DataFrame(list(zip(rappi_stores,rappi_cuisines_names,rappi_rating,rappi_eta)),columns = ['Store','Cuisine Type','Rating','Delivery Time'])

rappi_data

,Store,Cuisine Type,Rating,Delivery Time
0,[ Boston's ],"Pizza , Italiana",[ 4.2 ],100 min
1,[ Taquearte ],"Tacos , Mexicana",[ 4.3 ],100 min
2,[ Tacotlan ],Mexicana,[ 4.2 ],100 min
3,[ Last Slice ],Pizza,[ 4.5 ],100 min
4,[ McDonald's ],"Hamburguesas , Desayunos",[ 4.7 ],100 min
...,...,...,...,...
64,[ Starbucks ],"Café , Desayunos",[ 4.7 ],100 min
65,[ Taquería El Greco ],"Tacos , Árabe",[ 4.7 ],100 min
66,[ Hayito ],"Tacos , Árabe",[ 4.5 ],100 min
67,[ Lima Limón Frutería & Loncheria ],"Mexicana , Comida Rápida",[ 4.6 ],100 min


In [108]:
rappi_data.to_csv("/Users/miguel.d.ferrusca/Google Drive/Ironhack/Projects/web_scrapping/rappi_data.csv")